# A basic introduction to Data Science
**Disclaimer:** This is not a full introduction and unashamedly focusses on the areas I'm most interested in (optimisation). We are going to spend a small amount of time looking at howto extract data contained in text files. The focus is then on giving you an introduction to how some of the machine learning algorithms work under the hood (rather than just calling the existing implementations from a library). Overall this is very basic though, with a thorough treatment requiring at least a semester course. 

## Scenario
In this exerercise we are going to work through a fairly common scenario. After developing a couple of algorithms for solving a particular problem (or implementing some "standard" algorithms as well as your shiny new and hopefully improved algorithm), you test the algorithm on some instances and want to know which algorithm is "best"? Or which algorithm is best for what type of data? 

Here we are going to go through this type of analysis by focussing on alternative linear programming algorithms. So here are the elements that go into the analysis:
* The problem to be solved:  $\min c^T x$ subject to $A x = b,\ x \ge 0$ where $A$ is a $m\times n$ matrix, $x$ is a vector of decision variables and $b$ and $c$ are appropriate constant vectors. In practice we normally allow some variants (for example inequality constraints, arbitrary lower and upper bounds on the variables). 
* Algorithms: Three standard algorithms, *primal* simplex, *dual* simplex, and the *barrier* method, as implemented by the CPLEX solver library
* Instances: The [MIPLIB2010](http://miplib2010.zib.de/) "standard" test instances. The data sets are actually for mixed integer problems (where some variables are required to be binary or general integers), but for these tests we will throw away the binary requirements. There are 87 instances in this data set

To make things easy you will not be required to re-run all of the tests but are given the results from running each algorithm on each instance already pre-processed out of the log files from individual runs.



### Starting with just the data
To make things easy, here is a way to just start with the data you should have by now (you can also skip the above and just start from here).
As a reminder on using data frames, this [pandas selection tutorial](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c) provides a good summary on how to access rows,columns etc 

Also to make things easy there is a version of the data that has
* All timing (tick) information for the different algoritms removed
* Replace the min/max value of coefficients with the magnituded (log) of the range
* Scaled all columns to have entries in the range $[-1,1]$ (to have similar magnitudes)

You can load the raw data (including 'PrimalT', 'DualT' and 'BarrierT' performance fields) and the preprocessed features using the commands below:

In [11]:
import pandas
from urllib.request import urlopen
data = pandas.read_json(urlopen("http://users.monash.edu/~andrease/Files/Other/cplexLP.json"))
features = pandas.read_json(urlopen("http://users.monash.edu/~andrease/Files/Other/features.json"))
features.head(5) # show first few rows

D:\Anaconda\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


,VarCnt,VarFix,VarBox,ObjNZ,ConCnt,ConLess,ConEqual,ConNZ,RhzNZ,ConGreater,VarNneg,VarOther,VarFree,VarRng,ObjRng,ConRng,RhsRng
30n20b8,0.111769,1.0,0.067446,0.000019,0.000923,0.000779,0.000760,0.063267,0.000049,0.000000,0.0,0.0,0.0,0.116317,0.092743,1.304614e-01,-5.000000e-01
acc-tight5,0.008142,0.0,0.008138,0.000009,0.004890,0.001773,0.002338,0.024262,0.002313,0.010152,0.0,0.0,0.0,0.000000,-0.548711,-5.555556e-01,2.385606e-02
aflow40b,0.016589,0.0,0.016579,0.012847,0.002310,0.002186,0.000658,0.015732,0.000129,0.000000,0.0,0.0,0.0,0.100000,0.147614,1.108686e-01,4.771213e-02
air04,0.054145,0.0,0.054113,0.083861,0.001319,0.000000,0.006946,0.051596,0.001358,0.000000,0.0,0.0,0.0,0.000000,0.183693,-5.555556e-01,-5.000000e-01
app1-2,0.163402,0.0,0.080828,0.002505,0.085666,0.042635,0.112251,0.085247,0.044319,0.082518,1.0,0.0,0.0,0.000000,-0.548711,-2.413000e-07,-2.171000e-07


In [12]:
data

,VarCnt,VarFix,VarBox,ObjNZ,ConCnt,ConLess,ConEqual,ConNZ,RhzNZ,VarMin,...,ConMax,RhsMin,RhsMax,PrimalT,DualT,BarrierT,ConGreater,VarNneg,VarOther,VarFree
30n20b8,18380,7282,11098,2,576,486,90,109706,30,0,...,224.0,1.00000,1.0,65.00,54.04,296.40,0,0,0,0
acc-tight5,1339,0,1339,1,3052,1106,277,16134,1402,0,...,1.0,1.00000,4.0,699.24,409.07,723.36,1669,0,0,0
aflow40b,2728,0,2728,1364,1442,1364,78,6783,78,0,...,100.0,1.00000,10.0,7.54,24.45,11.87,0,0,0,0
air04,8904,0,8904,8904,823,0,823,72965,823,0,...,1.0,1.00000,1.0,2341.27,707.94,508.64,0,0,0,0
app1-2,26871,0,13300,266,53467,26600,13301,199175,26867,0,...,1.0,0.00001,1.0,5246.20,5208.74,5748.80,13566,13571,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
timtab1,397,13,384,128,171,0,171,829,165,0,...,60.0,0.00000,168.0,0.46,0.43,3.05,0,0,0,0
triptim1,30055,6,30028,1,15706,2054,9597,515436,4071,-9999999,...,6329.0,1.00000,1117.0,26136.61,26326.35,34572.17,4055,20,1,0
unitcal_7,25755,0,17136,9408,48939,25738,2907,127595,17512,0,...,1000.0,1.00000,9600.0,4082.17,637.14,3337.34,20294,8619,0,0
vpphard,51471,0,51471,46435,47280,46435,845,372305,46787,0,...,1.0,1.00000,5.0,6508.70,7119.07,9381.37,0,0,0,0


In [13]:
features

,VarCnt,VarFix,VarBox,ObjNZ,ConCnt,ConLess,ConEqual,ConNZ,RhzNZ,ConGreater,VarNneg,VarOther,VarFree,VarRng,ObjRng,ConRng,RhsRng
30n20b8,0.111769,1.000000,0.067446,0.000019,0.000923,0.000779,0.000760,0.063267,0.000049,0.000000,0.000000,0.000000,0.0,0.116317,0.092743,1.304614e-01,-5.000000e-01
acc-tight5,0.008142,0.000000,0.008138,0.000009,0.004890,0.001773,0.002338,0.024262,0.002313,0.010152,0.000000,0.000000,0.0,0.000000,-0.548711,-5.555556e-01,2.385606e-02
aflow40b,0.016589,0.000000,0.016579,0.012847,0.002310,0.002186,0.000658,0.015732,0.000129,0.000000,0.000000,0.000000,0.0,0.100000,0.147614,1.108686e-01,4.771213e-02
air04,0.054145,0.000000,0.054113,0.083861,0.001319,0.000000,0.006946,0.051596,0.001358,0.000000,0.000000,0.000000,0.0,0.000000,0.183693,-5.555556e-01,-5.000000e-01
app1-2,0.163402,0.000000,0.080828,0.002505,0.085666,0.042635,0.112251,0.085247,0.044319,0.082518,1.000000,0.000000,0.0,0.000000,-0.548711,-2.413000e-07,-2.171000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
timtab1,0.002414,0.001785,0.002334,0.001206,0.000274,0.000000,0.001443,0.005500,0.000272,0.000000,0.000000,0.000000,0.0,0.088543,0.178287,9.838067e-02,1.112655e-01
triptim1,0.182764,0.000824,0.182490,0.000009,0.025164,0.003292,0.080992,0.137135,0.006715,0.024665,0.001474,0.003817,0.0,0.350004,-0.548711,2.111853e-01,1.523832e-01
unitcal_7,0.156616,0.000000,0.104141,0.088608,0.078411,0.041253,0.024533,0.068231,0.028888,0.123443,0.635104,0.000000,0.0,0.134949,0.290872,1.666425e-01,1.991113e-01
vpphard,0.312994,0.000000,0.312806,0.437340,0.075753,0.074427,0.007131,0.116550,0.077179,0.000000,0.000000,0.000000,0.0,0.000000,-0.548711,-5.555556e-01,3.010300e-02


**Note:** For the much of the analysis we are only interested in the feature columns that _exclude_ the runtime (ticks) with the different methods (since these are an output rather than an input of the algorithms). We can get a copy of our data that excludes the runtime using
```python
features = data[[key for key in data if not key.endswith("T")]].copy()
```
We might also want to replace the various max/min elements with something more sensible such as the log of the max and min or log of the range (since it is more likely that this magnitutde has an impact than the absolute values).

In [2]:
import numpy as np

# back up the data so we can reload it easily when required
open("cplexLP.json","w").write(data.to_json())  # or pick your favourite data format...
print('Reload using: data = pandas.read_json(open("cplexLP.json","r"))')
# Preprocessing of data to make it more easily usable
data = pandas.read_json(open("cplexLP.json","r")) # reload data

features = data[[key for key in data if not key.endswith("T") and not key[-3:] in ["Max","Min"]]].copy()
for f in data:
    if f.endswith("Max"):
        features[f.replace("Max","Rng")] = np.log(np.maximum(
            data[f].values-data[f.replace("Max","Min")].values,np.ones(len(data[f].values))*1e-10))
features["ConNZ"] = np.sqrt(features["ConNZ"].values)
# scale everyting to be within [-1,1]
features /= np.max(features,axis=0)-np.min(features,axis=0)
open("features.json","w").write(features.to_json()) 
print('Reload using: features = pandas.read_json(open("features.json","r"))')

Reload using: data = pandas.read_json(open("cplexLP.json","r"))
Reload using: features = pandas.read_json(open("features.json","r"))


D:\Anaconda\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


## Visualising our data
This is quite high dimensional. How do we make sense of this?
Let's try to do a plot. For this we need to reduce the data to two dimensions. Enter PCA (Principal Component Analysis).
For most of the "machine learning" type algorithms in this workshop we will use the [scikit-learn](https://scikit-learn.org/stable/documentation.html) set of libraries. This inlcudes a [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) method. However so we can see more easily what is going on here, let's do it directly using the numpy matrix routines.

Useful functions:
* `data.values` gives the whole frame as a matrix. To get sensible results we want to (a) ignore the timing columns (those ending with T) and (b) replace the max/min of various values by the log of the range (difference between max and min);  (c) use the squareroot of ConNZ, since this is expected to be proportional to VarCnt * ConCnt
* We need to form the covariance matrix (subtract the column mean, multiply by the transpose and divide by n-1): $cov= \frac{1}{n-1}(X-I\bar x)^T (X-I\bar x)$. For this you need `np.mean` which takes an argument `axis=d` to take the mean in the d'th dimension, and `X.T` is the transpose of numpy matrix `X`. Or just call `numpy.cov` - but be careful not to get features & observations (columns and rows) mixed up
* `np.linalg.eig(X)` returns the (eigenvalues, eigenvectors) as vector/matrix respectively. 
* We only want to keep the two largest eigenvectors and plot the projection onto these two axes
* I'm assuming you are using matplotlib (`matplotlib.pyplot.scatter`) but you can use anything else for plotting

Question: Is the scaling of the different attributes proposed above sufficient? Shoud we do something more radical?

In [7]:
import matplotlib.pyplot as plt


# INSERT YOUR CODE YERE#

For alternative, more complicated ways to map data down to 2 dimensions there are plenty of other tools in sklearn library (look at [Manifold Learning](https://scikit-learn.org/stable/modules/manifold.html)). Looking at underlying algoirthms is well beyond the scope of this tutorial.  However you may want to have have a go at a few of these such as `TSNE` or `MDS`. Note that some of these manifold learning methods are randomised so won't return the same result each time.

Basic usage is something like
```Python
pts = TSNE(n_components=2).fit_transform(feastures)
# pts is an n x 2 matrix of the n points mapped into 2 dimensions
```

In [8]:
import numpy as np
from sklearn.manifold import TSNE, MDS,LocallyLinearEmbedding,SpectralEmbedding

# TEST YOUR CODE HERE #

## Unsupervised learning - clustering
In unsupervised learning we are looking for some structure without having any real idea of what is "correct". The simplest case of unsupervised learning is clustering - grouping data into clusters of points that are close together. 

The simplest way to do this is to apply the `cluster.KMeans` model from the `scklearn` library. See this [tutorial](https://scikit-learn.org/stable/tutorial/statistical_inference/unsupervised_learning.html) for more information.
Try splitting the data (based on the features table) into 3 clusters.

In [ ]:
from sklearn import cluster

### INSERT CODE HERE  ###

### Writing our own k-means algorithm
The basic algorithm is very simple - so simple that we can write our own
```
Pick an initial assignment of points to clusters (or an initial set of centres of the clusters)
Repeat until converged:
    Assign each point to the nearest centre (euclidean distance)
    Move the center of each cluster to the centroid (average of all points in the cluster)
```

This method is a heuristic: It is not guaranteed to give an optimal solution (the one with the least average distance of points to their centre) and it will converge to different solutions depending on the initial solution we start with.

Some usuful numpy funtions
* `np.random.randint(low,high,n)` returns array length n of integers in `range(low,high)`
* `np.mean(matrix,axis=d)` = array of means (along either columns or rows depending on axis
* `np.linalg.norm` = norm of an array
* `np.argmin(matrix,axis=d)` = like mean iterates over elements along just one axis and returns the minimum index
* `X[v > 0]` - returns a submatrix of X depending on which elements of `v` are positive. Whether `v` may be of length number of rows (to select whole rows) or same dimension as X (to select submatrix). Similarly `X[:,v>0]` would select submatrix based on columns where `v` is positive.

In [ ]:
import numpy as np
def kmeansHeur(data,p):
    """Given a n x m matrix with n points and m features 
    return list/np.array length n of labels (in range(p))  
    """
    data = np.array(data) # just to make convert from any other data type
    n,m = data.shape
    lbl = np.random.randint(0,p,n) # assign a random label from 0 to p-1
    ### INSERT YOUR CODE HERE ####
    return lbl

ncluster = 3
lblsHeur = kmeansHeur(features,ncluster) # fit model to data
print("Points per cluster=",[sum(1 for i in lblsHeur if i==k) for k in range(ncluster)])

## ADD CODE TO PLOT POINTS COLOURED BY LABEL:
# plt.scatter( <XVALUES>, <YVALUES>, c=lblsHeur)

This answer is probably not the same that you got wiht KMeans from sklearn. Which one is "right" or at least "better"? Need to formally define the objective function that we are trying to optimise. Essentially we are minimising the sum of squared norm distances:
$$\min_{c_k,C_k} \sum_k \sum_{i\in C_k} ||c_k - x_i||^2$$
Where $c_k$ and $C_k$ are the centre of cluster $k$ and the set of points in the cluster respectively. Each $x_i$ is a point of the data.

Compute the objective function for both your solution and the KMeans solution. Which one is better? Is either of these optimal (and how would we know?)

In [ ]:
def clusterObj(data,label):
    "Input: feature matrix (pts x features) and label for each feature."
    data = np.array(data)
    P = set(label) # unique labels
    centre = [np.mean(data[label==k],axis=0) for k in P]
    return sum(np.linalg.norm(data[i,:]-centre[k])**2  for i,k in enumerate(label))

# compare the two objectives: clusterObj(features,k_means.labels_),clusterObj(features,lblsHeur)

Note that our method is a randomised heuristic and the result depends on the starting point. Rerun your `kmeansHeur` repeatedly (say 500 times) and keep the best result. How does this compare to the result from a single run of `sklearn.cluster.KMeans` ?

In [ ]:
## INSERT YOUR CODE HERE ###


### Exact optimisation
Just to show that finding the best solution for clustering, even with such a small data set, is not trivial, here is a bit of code that uses the Mixed Integer Quadratic Programming solver from CPLEX. This can be ignored as it is not effective (need a much smarter formulation and/or custom algorithm to get a provably optimal solution). 
Documentation of the optimisation engines available via the [docplex](http://ibmdecisionoptimization.github.io/docplex-doc/) library (academic license availalbe and installed on maxima). The [examples](https://github.com/IBMDecisionOptimization/docplex-examples/tree/master/examples/cp/jupyter) provided by IBM include many jupyter notebooks.  An overview of [constraint types](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#module-docplex.cp.modeler) available is found here.

In [ ]:
# Mathematical programming optimisation model
import docplex.mp.model as MIP
D = np.array(features)
mdl = MIP.Model() # create mixed integer programming model
N,P = range(len(lblsHeur)),range(3)
rng = np.max(D,axis=0)-np.min(D,axis=0)
x = [mdl.binary_var_list(len(P)) for i in N] 
y = [mdl.continuous_var_list(D.shape[1],lb=0,ub=rng) for i in N]
z = [mdl.continuous_var_list(D.shape[1],lb=np.min(D,axis=0),ub=np.max(D,axis=0)) for k in P] # center
# assign each point to a cluster with each cluster non-empty
for i in N: mdl.add( mdl.sum(x[i][k] for k in P) == 1)
for k in P: mdl.add( mdl.sum(x[i][k] for i in N) >= 1)
Mn,Mx = np.min(D,axis=0),np.max(D,axis=0)
for i in N:
    for k,M in enumerate(rng):
        for j in P:
            mdl.add( y[i][k] >= z[j][k] - D[i,k] - (Mx[k]-D[i,k])*(1-x[i][j]))
            mdl.add( y[i][k] >= D[i,k] - z[j][k] - (D[i,k]-Mn[k])*(1-x[i][j]))
#F = range(len(rng))
#for i in N:
#    for j in N[i+1:]:
#        for k in P:
#            for f in F:
#                mdl.add(y[i][f]+y[j][f] >= abs(D[i,f]-D[j,f])*(x[i][k]+x[j][k]-1))
mdl.minimize(
    mdl.sum( y[i][k]**2 for k in range(len(rng)) for i in N)
)
CPXparam = mdl.context.cplex_parameters
CPXparam.timelimit = 20
CPXparam.threads = 2
#for p in [CPXparam.timelimit, CPXparam.threads]: print(p.get_qualified_name()," = ",p.get())
mdl.solve(log_output=True) # log_output = write to screen

How do we make this go faster?  Perhaps pass in a solution we have found already - just need to define the binary variables. Essential functions
* `soln = docplex.mp.solution.SolveSolution(mdl)` create a solution object for the model
* `soln.add_var_value(x[i][j],1)` sets the given variable to 1
* `soln.check_as_mip_start()` Is this a valid solution?
* `mdl.add_mip_start(soln)` Use the solution as a warm start for the optimisation
* `mdl.solve()` start/continue the optimisation

In [ ]:
import docplex
### INSERT CODE HERE ####

We can arbitrarily assign at least one point to a cluster - since renumbering the clusters gives us equivalent solutions, this just breaks the symmetry.
Furthermore the two points furthest appart are definitely going to be in different clusters. So assign these two points to different clusters (actual cluster numbers should match the heuristic solution)

In [ ]:
maxD,max_i,max_j = max( (np.linalg.norm(D[i,:]-D[j,:]),i,j) for i in N for j in N[i+1:])
print("Max distance %d <-> %d = %f"%(max_i,max_j,maxD))
mdl.add(x[max_i][lbls[max_i]]==1)
mdl.add(x[max_j][lbls[max_j]]==1)
mdl.add_mip_start(soln)
CPXparam.mip.strategy.heuristicfreq=1
#CPXparam.mip.strategy.rinsheur=1
soln = mdl.solve(log_output=True)

Time to give up on this idea - it might well run for hours without finding a good solution.

## Supervised Learning - Classification with Support Vector Machines
We might want to figure out which algorithm works fastest for a given problem. That is, given a linear program and its characteristics, can we decide which algorithm to use to get the solution as fast as possible. Support vector machines are one model for such a classification task. It supposes that there is a _linear_ function of the factors that determines which of 2 classes a point belongs to. That is, that there exists a set of weights $w_i$ and constant $W$ such that
$\sum_i w_i f_i < W$ if one algorithm is faster and $\sum_i w_i f_i > W$ when the other method is faster where $f_i$ are the features of the instance we are interested in solving. How do we decide what the $w_i$ (and $W$) should be? This is supervised learning where we are using existing _training data_ to fit the parameters. 

Let's start by trying to work out for which instances the primal vs dual simplex method is faster. We have `primalT` and `dualT` that tells us which is faster for the test data. So we want to find a vector $w$ and constant $W$ such that
$\sum_i w_i f_{ki} \le W-1$ if `PrimalT` < `DualT` and $\sum_i w_i f_{ki} \ge  W+1$ otherwise. (Where $f_{ki}$ is the i-th feature of instance k.  Finding such a set of $w_i$ and $W$ is just a linear programming problem. Some things we need to consider is: The limit of 1 is just to ensure we get some minimum separation (we could pick any number as multiplying each row by a constant would not change the problem other than to increase the non-zero difference between the LHS and W
* If the instances can be separated, we want to make the 2 hyperplanes (defined by the $\pm1$ on the right hand side) as far apart as possible(so that we get a clean separation). The separation distance is $2/||w||$. So maximising the separation is equivalent to $\min ||w||^2$ which gives us a quadratic program with linear constraints.
* It may not be possible to separate the points cleanly into two groups with a single hyperplane. Hence we might want to penalise any point that is mis-classified, perhaps based on how far it is away from the hyperplane. Let $v_k$ be the distance that point $k$ is away from the hyperplane then we might want to $\min ||w||^2 + \sum_k v_k$ with $\sum_i w_i f_{ki} \le W-1+v_k$,
* We can put the two objectives together - by noting that maximisation is the same as minimising the negative and placing greater priority on minimising violations:
$\min ||w||^2 + \alpha \sum_k v_k$ subject to $\sum_i w_i f_{ki} \le W+v_k-s$ for $k$ such that the primal solution is better (with correspoding $\ge$ constraints for the other points). Here $\alpha$ is an arbitrary weight that provides a trade-off between violations (misclassifications) and the distance separating the hyperplanes.
* Some variants of this are possible. For example we could replace the$||w||^2$ term by $\max_i |w_i|$ (for example) which can be written in a linear program, or even drop it entirely. When there are violations these are minimised  if $||w||$ is minimised so we may not need this.

Hence for an initial experiment we want to set up an SVM training function that takes our features as input data together with the list instances for which `data['PrimalT'] < data['dualT']` (this python expression returns a boolean column). Let `I` be those instances and `IC` be the complement. The we want to solve the following linear program:
$$\min_{v,w,W} \sum_{i\in I\cup IC} v_i$$
$$s.t.\ \sum_{k\in F} D_{if} w_f \le W - 1 + v_i\quad\forall\ i\in I$$
$$\quad \sum_{k\in F} D_{if} w_f \ge W + 1 - v_i\quad\forall\ i\in IC$$
$$ v_i \ge 0\quad\forall\ i$$
Here $D_{if}$ is the data for instance $i$ and feature $f$ out of the set of features $F$.

How to set this LP up using the CPLEX solver:
* `import docplex.mp.model as MP`
* `mdl = MP.Model()` create a mathematical programming (optimisation) model 
* `x = mdl.binary_var_list(n)` create list of variables `x[0]`...`x[n-1]` that are all in {0,1} - not needed here
* `x = mdl.continuous_var_list(n,lb=-mdl.infinity,ub=3*np.ones(n))` create list of variables $-\infty < x[i] \le 3$ for i=0,...,n-1
* `mdl.add( mdl.sum(x[i] for i in range(n)) == 1)` add the constraint $\sum_{i=0}^{n-1} x_i=1$
* `mdl.minimize( x[0]+x[1]**2)` set the objective (linear + quadratic term). Note: CPLEX handles linear and quadratic programming problems but not more general non-linear optimisation problems
* `CPXparam = mdl.context.cplex_parameters` access parameters - complete list of [CPLEX parameters](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.8.0/ilog.odms.cplex.help/CPLEX/Parameters/topics/introListAlpha.html). It may be a good idea to set `CPXparam.timelimit=60` seconds (to stop it running too long if the problem is not set up correctly), `CPXparam.threads=1`. This model should run in no more than a second though.
* `solution = mdl.solve(log_output=True)` does the optimisation and shows logs some information to the screen as it runs. `solution == None` if the solver fails to obtain a solution.
* `mdl.get_solve_status()` gives the final solution status (as a string)
* `solution.get_value(x[0]+x[1])` would return the final value of `x[0]+x[1]` in the solution. Alternatively you can also use `solution[x[2]]` to get the value of `x[2]` in the solution. (More informmation on the solution class see [SolveSolution](http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.solution.html#docplex.mp.solution.SolveSolution))



In [ ]:
# Mathematical programming optimisation model
import docplex.mp.model as MP

def SVM(features,label):
    """features: the usual matrix of points x features
    label: a list/array (length number of points) of bools/integers that
           identifies the points to be separated from the rest
    Returns: array length n.features+1 of w_i plus the constant W"""
    label = np.array(label) # to make sure we are not dealing with DataFrames
    #### INSERT YOUR CODE HERE #####
    return [solution.get_value(w[f]) for f in F]
soln = SVM(features,data['PrimalT']<data['DualT'])
print("Weights =",soln)

What can we do to improve the fit? How to deal with non-linear separations? Try adding additional "features" that capture the non-linear effects. For example we might expect that the effectiveness depends on the size or density of the constraint matrix (density = 'ConNZ' / ('VarCnt' x 'ConCnt')). Create an extended feature set and try again.
The idea here is that if we have say a two dimensional set of features (x,y for each point) and we wanted to separate those points that are inside an elipse centered at the origin from those outside, then we can find such a separating elipse by including features $x^2$ and $y^2$ with the optimisation choosing some combination $w_x x^2 + w_y y^2 \le W$ giving us an elipse while still solving a lienar problem (since the $x^2$ is just a constant coefficent for the variable $w_x$ in the optimisation).

Try creating some additional features. Remember for `DataFrames` we can easily create new columns by doing arithmetic with whole columns. For example `features['eqSq'] = features['ConEqual']**2` would create a new column `eqSq` that contains the number of equality constraints (`ConEqual`) squared. 

In [ ]:
### DEFINE SOME NEW FEATURES HERE ####

#### How well is our classifier doing?
To analyse the performance we should think about:
* Do we care about:
    1. how far we are on the wrong side of the line? (this is what we are optimising at the moment)
    2. how many instances are classified incorrectly?
    3. how much extra compute time we would incur if we used the minimum?
* How well it works for data it hasn't seen?
    
Below compute some alternative measures of the quality of the fit. Then address the second issue by using a random split of the instances to create 4 groups (of about 21 each). Then we used 3 parts as the "training" data to fit the SVM model, and the other group for testing to validate that the model is acutally OK on data that wasn't used for the training. By repeating this 4 times for each group we can compute a more accurate average performance of the approach on this type of data.

In [ ]:
def SVMviolation(feat,w,lower=data['PrimalT'],higher=data['DualT']):
    """Input: feat = matrix of features, w - list/array of weights (length features + 1), 
     higher/lower = performance measure
     Returns total violation of sum( w[i]*feat[i]) <= W or >= W depending on if lower[i] < or > higher[i]"""
    feat = np.array(feat) # just to make sure it is not a DataFrame
    v = feat.dot(np.array(w[:-1])) - w[-1]*np.ones(feat.shape[0])
    L = [ i for i,(low,high) in enumerate(zip(lower,higher)) if low < high]
    H = [ i for i,(low,high) in enumerate(zip(lower,higher)) if low > high]
    return sum( max(0,v[i]) for i in L)+sum(max(0,-v[i]) for i in H)
def SVMextraT(feat,w,lower=data['PrimalT'],higher=data['DualT']):
    """Input: feat = matrix of features, w - list/array of weights (length features + 1), 
     higher/lower = performance measure  (if feat * w < W we run the 'lower' method)
     Returns total extra time for running the slower algorithm"""
    v = np.array(feat).dot(np.array(w)[:-1])
    return sum(  max(0,low-high) if v[i] < 0 # extra time for low (Primal) algorithm if this is slower
               else max(0,high-low) # extra time to run the Dual (high) alg. if this is slower
        for i,(low,high) in enumerate(zip(lower,higher)) )
def SVMcount(feat,w,lower=data['PrimalT'],higher=data['DualT']):
    """Input: feat = matrix of features, w - list/array of weights (length features + 1), 
     higher/lower = performance measure  (if feat * w < W we run the 'lower' method)
     Returns total extra time for running the slower algorithm"""
    v = np.array(feat).dot(np.array(w)[:-1])
    return sum( 1 for i,(low,high) in enumerate(zip(lower,higher))
               if (low-high)*v[i] < 0 ) # mis-classified if low > high & v[i]<0 or low > high & v[i]>0

print("Results from training on whole data")
print("Effectiveness of solution: %.2f viol, %.2f ticks, %d misclassified"%(
    SVMviolation(features,soln),SVMextraT(features,soln),SVMcount(features,soln)))
print("Expanded feature set soln: %.2f viol, %.2f ticks, %d misclassified"%(
    SVMviolation(expFeat,expSoln),SVMextraT(expFeat,expSoln),SVMcount(expFeat,expSoln)))


Now partition your data into 4 subsets of about 21 instances each. Take each subset in turn as the test data and use the remainder to train (fit) the SVM. How well, on average, does this approach work for data it hasn't been trained on?

Note: while in general one might want to do the splitting randomly, this could lead to somewhat misleading results. The data sets are very variable in size & complexity, so it would make sense to try to split them a bit more systematically to ensure none of the groups only have very big/complex problems or only small trivial data sets. If you are feeling creative, make up a way to split the instances that deterministic or less random and more likely to be balanced.

In [ ]:
import random
# do things correctly using a subset of data for training only
def SVMtest(feat,nGroup=4,method=SVM,lower=data['PrimalT'],higher=data['DualT']):
    """Split features feat into nGroup groups, train using method, evaluate using lower/higher"""
    rows = [i for i in feat.index]
    random.seed(9999) # to make the split repeatable
    random.shuffle(rows)
    step = len(rows)//nGroup+1
    grps = [ rows[i:i+step] for i in range(0,len(rows),step)]
    print("#"*10,"Group lengths",[len(g) for g in grps])
    v,t,c = 0,0,0
    for g in range(nGroup):
        train = sum( (grps[i] for i in range(nGroup) if i!=g), []) # join all groups
        test = grps[g]
        w = method(feat.loc[train],lower[train] < higher[train])
        v += SVMviolation(feat.loc[test],w,lower[test],higher[test])
        t += SVMextraT(feat.loc[test],w,lower[test],higher[test])
        c += SVMcount(feat.loc[test],w,lower[test],higher[test])
    print("Performance with",nGroup,"groups:"
          "%.2f viol, %.2f ticks, %d misclassified"%(v,t,c))
    return v,t,c
print("Results when using original features: %.2f violation, %.2f ticks, %d misclassified"%
      SVMtest(features))
print("Results when using expanded features: %.2f violation, %.2f ticks, %d misclassified"%
    SVMtest(expFeat))

### Extension exercises
* We could modify our SVM approach to use an objective that better matches what we want to achieve (e.g. minimise additional computational effort from misclassified instances)
* Use the sklearn builtin method `LinearSVC` to implement a support vector machine. Documentation is available [here](https://scikit-learn.org/stable/modules/svm.html) The basic approach is:
```python
from sklearn import svm
mdl = svm.LinearSVC()
mdl.fit(X,y) # X = training data, y is -1 when PrimalT < DualT and +1 otherwise
mdl.predict(x) # predict outcome using test data x
```

# Appendix: Getting the data from log files

**Note:** This section provided only as background to give you some idea of where the data came from.

#### Obtaining data from log files
In an ideal world the data would already be sitting in a database or simple CSV file for you to read. However this may not always be possible. It is not uncommon for the data to be sitting in text files (perhaps the log files produced as the algorithms are run). Since such text files can get quite large, it makes sense to compress these. 

You can then process these log files using Python's string processing capabilities (including regular expressions) to extract what you really want. See the `TextProcessing.ipynb` exercise. Here we are going to skip this, but if you wanted to play with this aspects you can obtain the raw data as follows. That data is available as a zip file at http://users.monash.edu/~andrease/Files/Other/cplexLP.zip. You could just download this, unzip it and look at it - but we want to do things with python here. The basic library functions you need are
* `urllib` using the `urllib.request.urlopen("name.of.url/to-load")` function. Basic usage information is provided in this [how-to](https://docs.python.org/3.6/howto/urllib2.html?highlight=get%20url). It is probably easiest to just read this data and write it to file.
* `zipfile` module provides methods for reading (and writing) zip archives. The basic usage is to create `zip=ZipFile(<file>,'r')` archive, then use `zip.namelist()` to inspect the contents and `zip.open(name,'r')` to open the named file. Complete documentation is [here](https://docs.python.org/3/library/zipfile.html)
* Note that all log files are ASCII. To convert a binary (ASCII) string to the standard python unicode string use `.decode("utf-8")`

from urllib.request import urlopen
from zipfile import ZipFile

location = "http://users.monash.edu/~andrease/Files/Other/cplexLP.zip"
open("cplexLP.zip","bw").write(urlopen(location).read())
zipData = ZipFile("cplexLP.zip","r")
print("Zip file contains","; ".join(zipData.namelist()[:4]),"...")
if False: # this is too verbose - let's not print it
    print(zipData.open(zipData.namelist()[1],"r").read().decode("utf-8")) # arbitary example

#### Extracting Features
For each instance we want to extract the following information:
* The name (identifier) for the data set. This is encoded in the name of each file in the Zip archive (after the cplexLP/) or in the log file itself as part of the "Problem name"
* Characteristics or *features* of the instance to be solved. The log file contains some "statistics" of the problem such as the number of constraints and variables, number of non-zero coefficients, etc. See the CPLEX documentation on [displaying problem statistics](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.8.0/ilog.odms.cplex.help/CPLEX/GettingStarted/topics/tutorials/InteractiveOptimizer/displayingProbStats.html) for more informaiton
* The solution algorithm used: this is again shown in the "Problem name" or can be seen as an integer in the log file as "method for linear optimisation" (1=primal, 2=dual, 4=barrier)
* The time required using the solution algorithm: You could take the solution time in seconds, but for the analysis here using the "Deterministic time" in "ticks" is likely to be more useful as a way of assessing which algorithm is more efficient. Having said that, you may also want to extract the time in seconds. See this brief [discussion on ticks](http://www.thequestforoptimality.com/deterministic-behavior-of-cplex-ticks-or-seconds/) or CPLEX [documentation](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.8.0/ilog.odms.cplex.help/CPLEX/Parameters/topics/DetTiLim.html) for a little more information about what these mysterious ticks mean

In particular from the section of the log files that looks like this:
```
Objective sense      : Minimize
Variables            :   18380  [Fix: 7282,  Box: 11098]
Objective nonzeros   :       2
Linear constraints   :     576  [Less: 486,  Equal: 90]
  Nonzeros           :  109706
  RHS nonzeros       :      30

Variables            : Min LB: 0.000000         Max UB: 212.0000       
Objective nonzeros   : Min   : 51.00000         Max   : 100.0000       
Linear constraints   :
  Nonzeros           : Min   : 1.000000         Max   : 224.0000       
  RHS nonzeros       : Min   : 1.000000         Max   : 1.000000     

...lines deleted...

Deterministic time = 296.40 ticks  (956.13 ticks/sec)
```
We want to extract: the fields: VarCnt, VarFix,VarBox, ObjNZ,ConNZ, ConLess, ConEqual, ConNZ, RhsNZ, VarMin,VarMax,ObjMin,ObjMax,ConMin,ConMax,RhsMin,RhsMax,PrimalT (or DualT or BarrierT for the ticks)

#### Extracting the data

Have a go at extracting some/all of the data. Store the data in a pandas `DataFrame` object. See the [10 minute introduction](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) to Pandas if you have not come across this before. Essentially it is a "table like" data structure that makes it easy to manipulate data. 

We want to extract the following fields relating to Variables (Var), Constraints (Con) or the Objective function (Obj):  VarCnt, VarFix,VarBox, ObjNZ,ConNZ, ConLess, ConEqual, ConNZ, RhsNZ, VarMin,VarMax,ObjMin,ObjMax,ConMin,ConMax,RhsMin,RhsMax.
We also want the performance of the algorithm: PrimalT or DualT or BarrierT for the ticks.

For this exercise there is no need to get all of the fields that might be interesting them all (as getting them all can be a bit fiddly). To get a complete data set we will load a DataFrame of results extracted from these files below.

#### Storing it all as a Data frame

Have a go at extracting some/all of the data. Store the data in a pandas `DataFrame` object. See the [10 minute introduction](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) to Pandas if you have not come across this before. Essentially it is a "table like" data structure that makes it easy to manipulate data. 

We want to extract the following fields relating to Variables (Var), Constraints (Con) or the Objective function (Obj):  VarCnt, VarFix,VarBox, ObjNZ,ConNZ, ConLess, ConEqual, ConNZ, RhsNZ, VarMin,VarMax,ObjMin,ObjMax,ConMin,ConMax,RhsMin,RhsMax.
We also want the performance of the algorithm: PrimalT or DualT or BarrierT for the ticks.

For this exercise there is no need to get all of the fields that might be interesting them all (as getting them all can be a bit fiddly). To get a complete data set we will load a DataFrame of results extracted from these files below.

In [ ]:
import re,pandas
import numpy as np

probrex = re.compile(r"cplexLP/([^.]+)\.([a-z]+)\.log")
Instance = { probrex.match(d).group(1):{} for d in zipData.namelist()}
for d in zipData.namelist(): 
    m=probrex.match(d)
    Instance[m.group(1)][m.group(2)]=d
cntrex = re.compile(r"^(Variables|Linear)[^:]+: *(\d+) *(\[.*\])? *$")
floatmap={"all infinite":1e20,"all zero":0.0}
floatrex = r"-?\d+\.?\d*e?[-+0-9]*|all infinite|all zero" # cover the rare case of infinity
rangerex=re.compile(r"^ *(\w+).*: +Min.*: +("+floatrex+r") +Max.*: +("+floatrex+")")
rangemap={"Variables":"Var","Objective":"Obj","Nonzeros":"Con","RHS":"Rhs"}
nzmap = {"Objective":"ObjNZ", "Nonzeros":"ConNZ","RHS" : "RhzNZ" }
nzrex = re.compile(r"^ *("+"|".join(nzmap.keys())+r")[^:]*: *(\d+)$")
data = {} # list of records
for inst in Instance:
    res = {} # result
    continuation = ""
    for line in zipData.open(Instance[inst]["primal"],"r"): # read the statistics
        line = continuation + line.decode("utf-8") # in case line is continuing
        continuation = ""
        if "[" in line and not "]" in line:
            continuation = line.strip()
            continue
        m = cntrex.match(line)
        if m:
            base = "Var" if m.group(1)=="Variables" else "Con"
            res[base+"Cnt"] = int(m.group(2))
            if m.group(3):
                for name,val in [i.split(": ") for i in re.split(", *",m.group(3).strip("[]"))]:
                    res[base+name]=int(val)
            continue
        m = rangerex.match(line)
        if m:
            base = rangemap[m.group(1)]
            res[base+"Min"] = float(-floatmap[m.group(2)] if "all" in m.group(2) else m.group(2))
            res[base+"Max"] = float(floatmap[m.group(3)] if "all" in m.group(3) else m.group(3))
            continue
        m = nzrex.match(line)
        if m: res[nzmap[m.group(1)]] = int(m.group(2))
    # end loop over lines in primal data file
    for method in ["primal","dual","barrier"]:
        res[method.capitalize()+"T"]= float([
            line for line in zipData.open(Instance[inst][method]) # read every line
            if line.startswith(b"Deterministic time = ")][0] #only pick the (one and only) matching line
        .split()[3]) # 3rd field of line is ticks
    data[inst] = res
data = pandas.DataFrame.from_dict(data,orient="index") # could just do DataFrame(data) but that would give us the transpose of what we want
# Now just fix up missing values for the various counts and make sure they are integers
for key in data: 
    if key[:3] in ["Var","Con"] and key[-3:] not in ["Min","Max"]: 
        data[key].fillna(0,inplace=True)
        data[key] = data[key].astype(int) # integer type
data.head() # display data frame

In [ ]:
# let's just check for missing values
#data[np.logical_or.reduce(data.isnull(),axis=1)]
